In [1]:
from ortools.sat.python import cp_model
import pandas as pd
import numpy as np
from utility import JobPartialSolutionPrinter
from query_data import get_data_from_csv

In [2]:
data_dict = get_data_from_csv()
jobs_df = data_dict['jobs_df']
availability_df = data_dict['availability_df']
skills_df = data_dict['skills_df']
all_members = data_dict['all_members']
all_weeks = data_dict['all_weeks']
all_jobs = data_dict['all_jobs']
crucial_jobs = data_dict['crucial_jobs']
non_crucial_jobs = data_dict['non_crucial_jobs']

Testing Data Quality...
Checking if names appear in availability file
Checking if names appear in skills_mapping file
Checking that Names in both files match
Checking that Jobs in both files match
Data Quality is great!


In [3]:
# Creates the model
model = cp_model.CpModel()

# Creates job assignment variables.
shifts = {}
for m in all_members:
    for w in all_weeks:
        for j in all_jobs:
            shifts[(m, w, j)] = model.NewBoolVar(f"shift_m{m}_w{w}_j{j}")

# Each job is assigned to exactly one member per week.
for w in all_weeks:
    for j in crucial_jobs:
        model.AddExactlyOne(shifts[(m, w, j)] for m in all_members)

# Each non crucial job is assigned to at most one member per week.
for w in all_weeks:
    for j in non_crucial_jobs:
        model.AddAtMostOne(shifts[(m, w, j)] for m in all_members)

# Each member does at most one job per week.
for m in all_members:
    for w in all_weeks:
        model.AddAtMostOne(shifts[(m, w, j)] for j in all_jobs)

# Add availability constraints
for m in all_members:
    for w in all_weeks:
        if not availability_df.loc[m, w]: 
            for j in all_jobs:
                model.Add(shifts[(m, w, j)] == 0)

# Add skill constraints
for m in all_members:
    for j in all_jobs:
        if not skills_df.loc[m, j]: 
            for w in all_weeks:
                model.Add(shifts[(m, w, j)] == 0)
                
# Add constraint that members cannot be rostered thrice
for m in all_members:
    for w_idx in range(len(all_weeks) - 2):  # Ensure we don't go out of bounds
        for j in all_jobs:
            model.Add(
                shifts[(m, all_weeks[w_idx], j)] + shifts[(m, all_weeks[w_idx + 1], j)] + shifts[(m, all_weeks[w_idx + 2], j)] <= 2
            )


# Fairness constraint: Minimize variance in assignments
total_assignments = {}
for m in all_members:
    total_assignments[m] = model.NewIntVar(0, len(all_weeks) * len(all_jobs), f"total_assignments_{m}")
    model.Add(total_assignments[m] == sum(shifts[(m, w, j)] for w in all_weeks for j in all_jobs))

# Compute the mean assignments (rounded)
avg_assignments = len(all_weeks) * len(all_jobs) // len(all_members)

# Deviation from mean
deviation = {}
for m in all_members:
    deviation[m] = model.NewIntVar(0, len(all_weeks) * len(all_jobs), f"deviation_{m}")
    model.Add(deviation[m] >= total_assignments[m] - avg_assignments)
    model.Add(deviation[m] >= avg_assignments - total_assignments[m])

squared_deviation = {}
for m in all_members:
    squared_deviation[m] = model.NewIntVar(0, (len(all_weeks) * len(all_jobs)) ** 2, f"squared_deviation_{m}")
    model.AddMultiplicationEquality(squared_deviation[m], [deviation[m], deviation[m]])

# Minimize sum of squared deviations to reduce variance
model.Minimize(-sum(total_assignments[m] for m in all_members) * 10 + sum(squared_deviation[m] for m in all_members))
                


# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solver.parameters.enumerate_all_solutions = False

status = solver.Solve(model)#, solution_printer)

# Check if the optimal solution was found
if status == cp_model.OPTIMAL:
    print("\nOptimal solution found!")
    # Print the solution table
    solution_df = pd.DataFrame()
    solution_df["Job"] = all_jobs
    for w in all_weeks:
        week_list = []
        for j in all_jobs:
            job_filled = False
            for m in all_members:
                if solver.Value(shifts[(m, w, j)]):
                    week_list.append(m)
                    job_filled = True
            if not job_filled:
                week_list.append(np.nan)
        solution_df[f"{w}"] = week_list
    display(solution_df)
    solution_df.to_csv('data/final_solution.csv')
    # Print total assignments per member
    print("\nTotal Assignments per Member:")
    for m in all_members:
        total = solver.Value(total_assignments[m])  # Retrieve total assignments
        print(f"  - {m}: {total} jobs assigned")


elif status == cp_model.FEASIBLE:
    print("\nFeasible solution found, but not necessarily optimal.")
else:
    print("\nNo solution found.")




Optimal solution found!


,Job,2025-03-01,2025-03-08,2025-03-15,2025-03-22,2025-03-29,2025-04-05,2025-04-12,2025-04-19,2025-04-26,2025-05-03,2025-05-10,2025-05-17,2025-05-24
0,Coordinator(M),WeiXuan,WeiXuan,FamHanFeng,AndrewCheam,YeapYuXuan,AndrewCheam,YeapYuXuan,FamHanFeng,WeiXuan,AndrewCheam,YeapYuXuan,YeapYuXuan,WeiXuan
1,Livestream#1(M),YapMengHong,YapMengHong,NaN,YapMengHong,NaN,YapMengHong,YapMengHong,NaN,YapMengHong,YapMengHong,NaN,YapMengHong,YapMengHong
2,Sound#1(M),ShermanLee,WonSen,ShermanLee,WonSen,ShermanLee,WonSen,SingYe,SingYe,ShermanLee,ShermanLee,SingYe,WonSen,WonSen
3,Sound#2(M),WonSen,ShermanLee,SingYe,ShermanLee,SingYe,SingYe,WonSen,WonSen,SingYe,SingYe,WonSen,ShermanLee,ShermanLee
4,Projectionist(M),FredKwan,FredKwan,CheeFookSeng,HongTao,HongTao,IanMing,IanMing,CheeFookSeng,HongTao,HongTao,IanMing,FredKwan,CheeFookSeng
5,Coordinator(B),TimothyKwan,FamHanFeng,YeapYuXuan,TitusLeong,AndrewCheam,TitusLeong,WeiXuan,AndrewCheam,TimothyKwan,Jurina,Jurina,TimothyKwan,Jurina
6,Livestream#1(B),HoSiawEng,HungSweeGeok,MuTian,TengZhiQian,KooYuTang,MuTian,KooYuTang,TengZhiQian,HungSweeGeok,MuTian,KooYuTang,KooYuTang,MuTian
7,Livestream#2(B),HungSweeGeok,CheamHingGee,HungSweeGeok,KooYuTang,CheamHingGee,TengZhiQian,TengZhiQian,HungSweeGeok,CheamHingGee,TengZhiQian,CheamHingGee,MuTian,CheamHingGee
8,Sound#1(B),Jeffrey,ShawnHew,Jeffrey,JairusChong,JeremyFam,ShawnHew,JeremyFam,JairusChong,Jeffrey,ShawnHew,ShawnHew,JeremyFam,ShawnHew
9,Sound#2(B),JairusChong,Jeffrey,JairusChong,FamHanFeng,Jeffrey,JeremyFam,TitusLeong,TitusLeong,JeremyFam,TitusLeong,JairusChong,FamHanFeng,FamHanFeng



Total Assignments per Member:
  - FamHanFeng: 6 jobs assigned
  - ShawnHew: 5 jobs assigned
  - TitusLeong: 5 jobs assigned
  - Jeffrey: 5 jobs assigned
  - JeremyFam: 5 jobs assigned
  - MathiasChong: 0 jobs assigned
  - IanMing: 3 jobs assigned
  - Mavis: 0 jobs assigned
  - Jurina: 5 jobs assigned
  - Elly: 5 jobs assigned
  - MuTian: 5 jobs assigned
  - FuLiang: 0 jobs assigned
  - JairusChong: 5 jobs assigned
  - KooYuTang: 5 jobs assigned
  - WonSen: 9 jobs assigned
  - ShermanLee: 9 jobs assigned
  - FredKwan: 3 jobs assigned
  - CheeFookSeng: 3 jobs assigned
  - SingYe: 8 jobs assigned
  - TengZhiQian: 5 jobs assigned
  - HoSiawEng: 5 jobs assigned
  - TimothyKwan: 5 jobs assigned
  - AndrewCheam: 5 jobs assigned
  - YapMengHong: 9 jobs assigned
  - WeiXuan: 5 jobs assigned
  - CheamHingGee: 5 jobs assigned
  - HungSweeGeok: 5 jobs assigned
  - HongTao: 4 jobs assigned
  - YeapYuXuan: 5 jobs assigned
